In [2]:
import sys
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages


2024-03-24 15:32:53.211247: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 15:32:53.282774: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 15:32:53.786762: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split


In [4]:
from recommenders.evaluation.python_evaluation import (
    ndcg_at_k, precision_at_k, recall_at_k
)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
Pandas version: 2.2.1
Tensorflow version: 2.16.1


In [5]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 50
BATCH_SIZE = 256

SEED = 42

In [6]:
df = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating", "timestamp"]
)

INFO:recommenders.datasets.download_utils:Downloading https://files.grouplens.org/datasets/movielens/ml-100k.zip
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 4.81k/4.81k [00:01<00:00, 2.46kKB/s]


In [7]:
train, test = python_chrono_split(df, 0.75)

/opt/conda/envs/rec/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/opt/conda/envs/rec/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/opt/conda/envs/rec/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/opt/conda/envs/rec/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/opt/conda/envs/rec/lib/python3.9/si

In [8]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

In [9]:
train_file = "./train.csv"
test_file = "./test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

In [10]:
data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED)

INFO:recommenders.models.ncf.dataset:Indexing ./train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test_full.csv ...


In [12]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/opt/conda/envs/rec/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2024-03-24 15:35:46.779172: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-03-24 15:35:46.779187: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: pop-os
2024-03-24 15:35:46.779189: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: pop-os
2024-03-24 15:35:46.779273: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 525.89.2
2024-03-24 15:35:46.779279: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 525.89.2
202

In [13]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time))

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [1.32s]: train_loss = 0.259587 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [1.31s]: train_loss = 0.246305 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 30 [1.35s]: train_loss = 0.239579 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 40 [1.35s]: train_loss = 0.234690 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 50 [1.34s]: train_loss = 0.230666 


Took 67.6170 seconds for training.


In [14]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 2.6957 seconds for prediction.


In [19]:
user

[943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943,
 943

In [18]:
model.predict(user, item, is_list=True)

[0.9221864,
 0.94902754,
 0.0017495696,
 0.71527165,
 0.0026898242,
 0.6525583,
 0.48163116,
 0.011429842,
 0.5762642,
 0.66541284,
 0.97611666,
 0.92655176,
 0.53122723,
 0.8873805,
 0.09574927,
 0.61879027,
 0.22514711,
 0.95068,
 0.3367221,
 0.0018705969,
 0.001893425,
 0.08617288,
 0.00047257057,
 0.12654145,
 0.0030561194,
 0.7315797,
 0.050435964,
 0.00046537648,
 0.9394942,
 0.76193106,
 0.3939475,
 0.10197193,
 0.14895545,
 0.7219874,
 0.05256885,
 0.015398034,
 0.6907833,
 0.16034685,
 0.75984687,
 0.9673081,
 0.0014468885,
 0.024026552,
 0.70727575,
 0.87083405,
 0.01651732,
 0.35154247,
 0.07093646,
 0.38540414,
 0.701464,
 0.7859553,
 0.08957234,
 0.50949234,
 0.067959614,
 0.8160312,
 0.12075242,
 0.89714485,
 0.90923965,
 0.94205326,
 0.89869934,
 0.76074016,
 0.014050622,
 0.14480066,
 0.5701747,
 0.8878047,
 0.9248454,
 0.7910957,
 0.4467618,
 0.20667385,
 0.040904183,
 0.8731631,
 0.84292996,
 0.43928924,
 0.6354179,
 0.61164826,
 0.24682988,
 0.20998596,
 0.22315653,


In [17]:
test

,userID,itemID,rating,timestamp
73358,1,149,2.0,878542791
88893,1,88,4.0,878542791
59055,1,101,2.0,878542845
43438,1,110,1.0,878542845
14792,1,103,1.0,878542845
...,...,...,...,...
81005,943,450,1.0,888693158
92536,943,227,1.0,888693158
95003,943,230,1.0,888693158
94914,943,229,2.0,888693158


In [30]:
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

NDCG:	0.200148
Precision@K:	0.178155
Recall@K:	0.098838


In [32]:
?model.predict

Signature: model.predict(user_input, item_input, is_list=False)
Docstring:
Predict function of this trained model

Args:
    user_input (list or element of list): userID or userID list
    item_input (list or element of list): itemID or itemID list
    is_list (bool): if true, the input is list type
        noting that list-wise type prediction is faster than element-wise's.

Returns:
    list or float: A list of predicted rating or predicted rating score.
File:      /opt/conda/envs/rec/lib/python3.9/site-packages/recommenders/models/ncf/ncf_singlenode.py
Type:      method

In [37]:
?model.

Type:        OrderedDict
String form: OrderedDict([(1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11 <...>  (936, 935), (937, 936), (938, 937), (939, 938), (940, 939), (941, 940), (942, 941), (943, 942)])
Length:      943
File:        /opt/conda/envs/rec/lib/python3.9/collections/__init__.py
Docstring:   Dictionary that remembers insertion order

In [38]:
1

1